# Deep Learning Project - Orca Call Clustering 
Karlee Zammit - V00823093 

https://github.com/earthspecies/library/tree/main/orcas

https://debuggercafe.com/pytorch-pretrained-efficientnet-model-image-classification/

In [2]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.datasets import ImageFolder
from PIL import Image, ImageOps
import os.path, sys
import tensorflow as tf
import torchaudio
import torchaudio.transforms as T
import librosa
import matplotlib.pyplot as plt
import wave
from IPython.display import Audio
from torchvision.io import read_image
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from sklearn.model_selection import train_test_split
!pip install torch torchvision efficientnet_pytorch==0.7.0
from efficientnet_pytorch import EfficientNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Functions

In [3]:
# Plot and save the waveform from the wave file 
def plot_waveform(idx, waveform, sr, title="Waveform"):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sr

    figure, axes = plt.subplots(num_channels, 1)
    axes.plot(time_axis, waveform[0], linewidth=1)
    axes.grid(True)
    figure.suptitle(title)
    plt.show(block=False)
    #plt.savefig(r'C:\Users\kzammit\Repos\school\phys555\project-DL\orcas_classification\wavpngs\wav-' + str(idx) + '.png')
    plt.close()

In [4]:
# Plot and save the spectrogram from the wave file
#def create_spectrogram(idx, specgram, title=None, ylabel="freq_bin"):
def create_spectrogram(idx, specgram):
    #fig, axs = plt.subplots(1, 1)
    #axs.set_title(title or "Spectrogram (db)")
    #axs.set_ylabel(ylabel)
    #axs.set_xlabel("frame")
    #axs.set_ylim(0, 100)
    #print(specgram.shape)
    #imgf = np.flip(specgram, axis=0)
    transform=transforms.ToPILImage()
    #specgram=transform(specgram)
    specgram= transform(specgram)
    specgram = ImageOps.flip(specgram)
    #specgram.save(r"C:\Users\kzammit\Documents\PHYS555\orcas_classification\specs\spec-" + str(idx) + ".png")
    display(specgram)
    
    #im = Image.fromarray(librosa.power_to_db(specgram))
    #if im.mode != 'RGB':
    #    im = im.convert('RGB')
    #im.save("your_file.jpeg")
    #im = axs.imshow(librosa.power_to_db(specgram), origin="lower", aspect="auto")
    # librosa... is a numpy array, save that to a png file instead 
    #fig.colorbar(im, ax=axs)
    #plt.show(block=False)
    #plt.savefig(r'C:\Users\kzammit\Repos\school\phys555\project-DL\orcas_classification\spectrograms\spec-' + str(idx) + '.png')
    #plt.close()

In [5]:
# Crop the spectrogram figures and save as pngs
def crop(path, dirs):
    for item in dirs:
        fullpath = os.path.join(path,item)
        if os.path.isfile(fullpath):
            im = Image.open(fullpath)
            f, e = os.path.splitext(fullpath)
            # last one does the height
            # left top right bottom
            imCrop = im.crop((80, 58, 475, 425)) #corrected
            #imCrop.save(f + '-crop.png', "PNG", quality=300)
            display(imCrop)

In [6]:
def preprocess():
    
    transform =  transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], 
                             std=[0.229]),])
    return transform

#transforms.Normalize(mean=[0.485, 0.456, 0.406], 
#                     std=[0.229, 0.224, 0.225]),])

# Spectrogram Generation and Preprocessing

In [ ]:
# For each available wave file
for ii in range(59, 60):
    
    # In the audio folder 
    path = r'C:\Users\kzammit\Documents\PHYS555\orcas_classification\audio'
    
    # Get the file name and path
    file = '\\' + str(ii) + '.wav'
    fullfile = path + file

    # Set the sample to this file
    SAMPLE_WAV = fullfile

    # Load the wave with torchaudio 
    ORCA_WAVEFORM, SAMPLE_RATE = torchaudio.load(SAMPLE_WAV)

    # Plot the original waveform 
    plot_waveform(idx=str(ii), waveform=ORCA_WAVEFORM, sr=SAMPLE_RATE, title="Original waveform")
    Audio(ORCA_WAVEFORM.numpy(), rate=SAMPLE_RATE)

    # Set the spectrogram parameters 
    n_fft = 2000
    win_length = 2000
    hop_length = 100

    # Define spectrogram settings using torch audio transforms  
    spectrogram = T.Spectrogram(n_fft=n_fft, win_length=win_length, hop_length=hop_length, center=True, pad_mode="reflect", power=2.0)
    
    # Calculate the spectrogram 
    spec = spectrogram(ORCA_WAVEFORM)
    
    # Plot and save the spectrogram
    create_spectrogram(idx=str(ii), specgram=spec[0])

# Load in Data

In [7]:
class OrcaImageDataset(Dataset):
    
    def __init__(self, annotations_file, img_dir, transform, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [8]:
#orca_dataset = OrcaImageDataset(annotations_file, img_dir, transform=transforms.Grayscale())
#input_height = 367
#input_width = 395
#perm_val = input_height*input_width
#resize = transforms.Resize(size=(input_height, input_width), antialias=True) # aspect ratio is important, and center crop
#orca_dataset = OrcaImageDataset(annotations_file, img_dir, transform= transforms.Compose([transforms.Lambda(lambda x: x[:3]), resize, transforms.ToTensor()])) # drop the alpha channel from the image
#orca_dataset = OrcaImageDataset(annotations_file, img_dir, transform= transforms.Compose([transforms.Lambda(lambda x: x[:3]), resize, transforms.Grayscale(num_output_channels=1)]))
#orca_dataset = OrcaImageDataset(annotations_file, img_dir, transform= transforms.Compose([transforms.Lambda(lambda x: x[:3]), resize]))# drop the alpha channel from the image

annotations_file = r"C:\Users\kzammit\Documents\PHYS555\orcas_classification\annotations-edited.csv"
img_dir = r"C:\Users\kzammit\Documents\PHYS555\orcas_classification\specs"
transforms_eff = preprocess()
orca_dataset = OrcaImageDataset(annotations_file, img_dir, transform=transforms_eff)

In [9]:
labels = orca_dataset.img_labels
#labels = labels.loc[:,"label"]

In [10]:
xtrain_data, test_data, ytrain_labels, test_labels = train_test_split(
    orca_dataset, labels, test_size=0.2, random_state=21)

#  split a validation set from the training set
train_data, val_data, train_labels, val_labels = train_test_split(
    xtrain_data, ytrain_labels, test_size=0.2, random_state=22)

In [11]:
print('The number of samples for training is ' + str(len(train_data)))
print('The number of samples for validation is ' + str(len(val_data)))
print('The number of samples for testing is ' + str(len(test_data)))

The number of samples for training is 380
The number of samples for validation is 95
The number of samples for testing is 119


In [12]:
batch_size = 1
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Data Visualization

In [ ]:
# Crop the spectrograms and save as images 
path = r"C:\Users\kzammit\Documents\PHYS555\orcas_classification\example_crop"
dirs = os.listdir(path)
crop(path, dirs)

# the initial size of the image 

In [ ]:
print(orca_dataset.img_labels)

In [ ]:
print('The shape of tensor for 60th image in train dataset: ', orca_dataset[60][0].shape)
print(orca_dataset)

In [ ]:
print(orca_dataset[60][0])

In [ ]:
transform = transforms.ToPILImage()
plt.figure(figsize=(16, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    image, _ = train_loader.dataset.__getitem__(i)
    #print(image.shape)
    image = transform(image)
    plt.imshow(image)
    plt.axis('off');
    #print(image.size)

# Models

## CNN Autoencoder

In [ ]:
# https://medium.com/analytics-vidhya/implementing-under-over-autoencoders-using-pytorch-4ddaf458947e
class CNN_autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(3,3)),  # from 1 ---> 4, size = 26x26
            #nn.ReLU(),
            nn.Conv2d(in_channels=4, out_channels=16, kernel_size=(3,3)),  # from 4 --> 16, size = 24x24
            #nn.Conv2d(in_channels=4, out_channels=1, kernel_size=(3,3)),  # from 4 --> 16, size = 24x24
            #nn.ReLU(),     #
            nn.MaxPool2d(2,2),                                              # size=12x12
            #nn.Tanh()
            #nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=16, out_channels=4, kernel_size=(3,3)),  # 16 --> 4, size=14*14
            #nn.ConvTranspose2d(in_channels=1, out_channels=4, kernel_size=(3,3)),  # 16 --> 4, size=14*14
            #nn.ReLU(),
            nn.ConvTranspose2d(in_channels=4, out_channels=1, kernel_size=(2,2), stride=2),  # 4 --> 1, size=28*28
            #nn.ReLU(),
            nn.Tanh()                  # TanH is recommended as the output image pixel intensities have to lie between -1 & 1
        )

    def forward(self, x):
        for l in self.encoder:
            x = l(x)
            lspace = x
        for l in self.decoder:
            x = l(x)
        return x, lspace

In [13]:
def train_test_model(model, train_loader, val_loader, n_epochs, optimizer):
    #transform = transforms.ToPILImage()
    criterion = nn.MSELoss()
    optimizer = optimizer
    model = model.train()
    train_losses = []
    val_losses = []
    
    for epoch in range(n_epochs):
        train_batch_losses = []
        val_batch_losses = []
        for data in train_loader:
            img, _ = data
            img    = img.to(device)
            output, lspace = model(img)
            loss = criterion(output, img.data)
            #************************ backward *************************
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_batch_losses.append(loss.item()) 
            
        model = model.eval()
        for data in val_loader:
            output, lspace = model(img)
            loss = criterion(output, img.data)
            val_batch_losses.append(loss.item())
            
        train_loss = np.mean(train_batch_losses)
        val_loss = np.mean(val_batch_losses)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
            
        # ***************************** log ***************************
        if epoch % 100 == 0:
            print(f"epoch [{epoch + 1}/{num_epochs}], Train loss:{train_loss: .4f} Valid:{val_loss: .4f}")
    
    ax = plt.figure().gca()
    ax.plot(train_losses)
    ax.plot(val_losses)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'val'])
    plt.title('Loss monitoring')
    plt.show()
    
    return model.eval(), output, lspace

In [ ]:
conv_model = CNN_autoencoder().to(device)
num_epochs = 100
optimizer = torch.optim.Adam(conv_model.parameters(), lr=1e-3)
#output, lspace = train_model(conv_model, train_loader, num_epochs)
model, output, lspace = train_test_model(conv_model, train_loader, val_loader, num_epochs, optimizier)

In [ ]:
# Plot the latent space and reconstructed spectrogram
transform = transforms.ToPILImage()
img = transform(lspace[0][0])
display(img)
img_o = transform(output[0][0])
display(img_o)

In [ ]:
print(lspace.size())
print(output.size())

## Efficient Net Pretrained Encoder, Convolutional Decoder

In [14]:
# Fake it till u make it bb
def expand_greyscale_image_channels(grey_image):
    grey_image = grey_image.to('cpu')
    grey_image_arr = np.array(grey_image)
    grey_image_arr = np.expand_dims(grey_image_arr, -1)
    grey_image_arr_3_channel = grey_image_arr.repeat(3, axis=-1)
    grey_image_arr_3_channel = torch.from_numpy(grey_image_arr_3_channel)
    grey_image_arr_3_channel = grey_image_arr_3_channel.to(device)
    
    print(grey_image_arr_3_channel.shape)
    grey_image_arr_3_channel = torch.squeeze(grey_image_arr_3_channel)
    print(grey_image_arr_3_channel.shape)
    grey_image_arr_3_channel = grey_image_arr_3_channel.T
    print(grey_image_arr_3_channel.shape)
    grey_image_arr_3_channel = grey_image_arr_3_channel.unsqueeze(0)
    print(grey_image_arr_3_channel.shape)
    return grey_image_arr_3_channel

In [54]:
class PreT_autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = EfficientNet.from_pretrained('efficientnet-b0', include_top=False)

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1280, 512, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 224, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(224, 1, kernel_size=4, stride=2, padding=1),
            nn.ReLU()
        )
        

    def forward(self, x):
        x = expand_greyscale_image_channels(x)
        x = self.encoder.extract_features(x)
        #print(x.shape)
        lspace = x
        for l in self.decoder:
            x = l(x)
            #print(x.shape)
        print(x.shape)
        return x, lspace

In [55]:
preT_model = PreT_autoencoder().to(device)
num_epochs = 100
#output, lspace = train_model(conv_model, train_loader, num_epochs)
optimizer = torch.optim.Adam(preT_model.parameters(), lr=1e-3)
model, output, lspace = train_test_model(preT_model, train_loader, val_loader, num_epochs, optimizer)

Loaded pretrained weights for efficientnet-b0
torch.Size([1, 1, 224, 224, 3])
torch.Size([224, 224, 3])
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 1, 112, 112])


RuntimeError: The size of tensor a (112) must match the size of tensor b (224) at non-singleton dimension 3

Project minimum: Load a better model, U Map of the latent space, clusters, plot with labels of the call types 
fix the encoder part with a pretrained network, learn the decoder

self.resnet.fc = nn.Identity() # remove the fully connected laye
        
add a decoder that's not pretrained 
    
try to use a powerful pretrained encoder to a basic one i made (efficient net v2) 

# my_spe = autoencoder(data)
# make list of my spec . features, send that to umap, plot this representation 
# 

In [ ]:
def train_model(model, train_loader, n_epochs):
    #transform = transforms.ToPILImage()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(conv_model.parameters(), lr=1e-3)
    for epoch in range(n_epochs):
        for data in train_loader:
            img, _ = data
            img    = img.to(device)
            output, lspace = model(img)
            loss = criterion(output, img.data)
            #************************ backward *************************
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # ***************************** log ***************************
        if epoch % 100 == 0:
            print(f'epoch [{epoch + 1}/{num_epochs}], loss:{loss.item(): .4f}')
    return output, lspace

In [ ]:
model = models.efficientnet_b0(pretrained=True)models.efficientnet_b0(pretrained=True)